The aim of this blog post is to test that we can correctly reconstruct a simple signal. And also to better understand the data of each stage.

Let's test first by randomly selecting between [0, 0] and [1, 1] vectors.

In [1]:
from src.var_processor.vpu import VPU, VPUBinary
import random
import numpy as np

In [2]:
random.random()

0.46799349226432885

In [3]:
def rand_same():
    a = np.empty([2, 1])
    a.fill(np.random.randint(2))
    return a

In [4]:
def rand_diff():
    a = np.zeros([2, 1])
    index = np.random.randint(2)
    a[index] = 1
    return a

In [5]:
print(rand_same(), rand_same().shape)

[[1.]
 [1.]] (2, 1)


In [6]:
print(rand_diff(), rand_diff().shape)

[[1.]
 [0.]] (2, 1)


In [7]:
vpu = VPU(2)
print(vpu.cu.covariance, vpu.pi.ev)
for _ in range(0, 1000):
    vpu.update_cov(rand_same())
    vpu.pi.iterate(cov=vpu.cu.covariance)
print(vpu.cu.covariance, vpu.pi.ev)

[[0. 0.]
 [0. 0.]] [[0.81341381]
 [0.58168546]]
[[0.24586054 0.24586054]
 [0.24586054 0.24586054]] [[0.70710678]
 [0.70710678]]


In [8]:
print(vpu.pi.eigenvalue, vpu.pi.eigenvector)

[[0.49172109]] [[0.70710678]
 [0.70710678]]


In [9]:
np.sqrt(vpu.pi.eigenvalue)

array([[0.70122827]])

In [10]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))

array([[0.49584326],
       [0.49584326]])

Does the above expression thus give you something to do with the probability for the vector?

In [11]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[0.96384326],
       [0.96384326]])

We can use this to same with x = 1 or 0? Or x = -1, or 1?

In [12]:
print(
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean,
    -1*np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
)

[[0.96384326]
 [0.96384326]] [[-0.02784326]
 [-0.02784326]]


Yes - x = -1 or 1 or random value within this range.

In [26]:
# Adapt to test

vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_same())
    vpu.pi.iterate(cov=vpu.cu.covariance)
# Check all values of covariance matrix are the same
print(vpu.cu.covariance, vpu.cu.covariance[0])
assert np.allclose(vpu.cu.covariance, vpu.cu.covariance[0])
# Check eigenvector has values of root 2
print(vpu.pi.eigenvector, 1/np.sqrt(2))
assert np.allclose(vpu.pi.eigenvector, 1/np.sqrt(2))
sample_1 = np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
sample_minus1 = -1*np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
assert np.allclose(sample_1, np.array([1, 1]), rtol=0.05, atol=0.05)
assert np.allclose(sample_minus1, np.array([0, 0]), rtol=0.05, atol=0.05)

[[0.24691728 0.24691728]
 [0.24691728 0.24691728]] [0.24691728 0.24691728]
[[0.70710678]
 [0.70710678]] 0.7071067811865475


In [24]:
1/np.sqrt(2)

0.7071067811865475

In [13]:
vpu = VPU(2)
print(vpu.cu.covariance, vpu.pi.ev, sep="\n", end="\n\n")
for _ in range(0, 1000):
    vpu.update_cov(rand_diff())
    vpu.pi.iterate(cov=vpu.cu.covariance)
print(vpu.cu.covariance, vpu.pi.ev, vpu.pi.eigenvalue, sep="\n", end="\n\n")
print(
    vpu.cu.mean,
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue)), 
    np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean,
    sep="\n",
    end="\n\n"
)

[[0. 0.]
 [0. 0.]]
[[0.79083108]
 [0.61203448]]

[[ 0.24667776 -0.24667776]
 [-0.24667776  0.24667776]]
[[ 0.70710678]
 [-0.70710678]]
[[0.49335551]]

[[0.486]
 [0.514]]
[[ 0.49666665]
 [-0.49666665]]
[[0.98266665]
 [0.01733335]]



How would we get a sample of [0, 1] if x = 0 then we just have mean. Ah - x needs to be between -1 and 1.

In [14]:
# Forward Pass
a = rand_diff()
print(a, np.dot(vpu.pi.eigenvector.T, a))

[[0.]
 [1.]] [[-0.70710678]]


In [15]:
np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[0.98266665],
       [0.01733335]])

In [16]:
-1*np.dot(vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean

array([[-0.01066665],
       [ 1.01066665]])

In [34]:
# Adapt to test

vpu = VPU(2)
for _ in range(0, 1000):
    vpu.update_cov(rand_diff())
    vpu.pi.iterate(cov=vpu.cu.covariance)
# Check diagonal values of covariance matrix are the same
# Use https://docs.scipy.org/doc/numpy/reference/generated/numpy.diagonal.html
print(vpu.cu.covariance[0], -1*vpu.cu.covariance[-1])
assert np.allclose(vpu.cu.covariance[0], -1*vpu.cu.covariance[-1])
# Check eigenvector has values of root 2
print(vpu.pi.eigenvector, 1/np.sqrt(2))
assert np.allclose(np.abs(vpu.pi.eigenvector), 1/np.sqrt(2))
# Check different signs
assert np.allclose(vpu.pi.eigenvector[0], -1*vpu.pi.eigenvector[1])
sample_1 = np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
sample_minus1 = -1*np.dot(
        vpu.pi.eigenvector, np.sqrt(vpu.pi.eigenvalue))+vpu.cu.mean
print(sample_1, np.flipud(sample_minus1))
assert np.allclose(sample_1, np.flipud(sample_minus1), rtol=0.1, atol=0.1)

[ 0.24607419 -0.24607419] [ 0.24607419 -0.24607419]
[[ 0.70710678]
 [-0.70710678]] 0.7071067811865475
[[ 1.02705866]
 [-0.02705866]] [[0.96505866]
 [0.03494134]]
